In [1]:
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 
import pyproj

In [2]:
cal_coords_list =[]
cal_y_list =[]
cal_X_list =[]
delt_stwr_intervel =[0.0]

In [3]:
csvFile = open("beijing_all_20160423.csv", "r")
df = pd.read_csv(csvFile,header = 0,
                 skip_blank_lines = True,
                 keep_default_na = False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 17 columns):
Unnamed: 0    384 non-null int64
lon           384 non-null float64
lat           384 non-null float64
temp          384 non-null float64
hpa           384 non-null float64
wet           384 non-null float64
speed         384 non-null float64
dir           384 non-null float64
mm            384 non-null float64
height        384 non-null float64
t_time        384 non-null float64
pm25          384 non-null object
pm25_24       384 non-null object
pm10          384 non-null object
pm10_24       384 non-null object
aqi           384 non-null float64
time          384 non-null float64
dtypes: float64(12), int64(1), object(4)
memory usage: 51.1+ KB


In [4]:
df = df.drop(columns=['pm25', 'pm25_24','pm10','pm10_24'])
df['aqi' ]=pd.to_numeric(df['aqi'])

df = df.sort_values(['time']) 
all_data = df.values
#remove nan in all_data
all_data = all_data[~np.isnan(all_data).any(axis=1)]
all_data = all_data[:,1:]

In [5]:

tick_time = all_data[0,-1]
cal_coord_tick = []
cal_X_tick =[]
cal_y_tick =[]

time_tol = 1.0e-7

lensdata = len(all_data)
for row in range(lensdata):
    cur_time = all_data[row,-1]
    if(abs(cur_time-tick_time)>time_tol):
        cal_coords_list.append(np.asarray(cal_coord_tick))
        cal_X_list.append(np.asarray(cal_X_tick))
        cal_y_list.append(np.asarray(cal_y_tick))
        delt_t = cur_time - tick_time
        delt_stwr_intervel.append(delt_t) 
        tick_time =cur_time
        cal_coord_tick = []
        cal_X_tick =[]
        cal_y_tick =[]
    coords_tick = np.array([all_data[row,0],all_data[row,1]])
    cal_coord_tick.append(coords_tick)

    x_tick = np.array([all_data[row,2],all_data[row,3],all_data[row,4],all_data[row,5],all_data[row,6],all_data[row,8]])
    cal_X_tick.append(x_tick)
    y_tick = np.array([all_data[row,-2]])
    cal_y_tick.append(y_tick)


In [6]:
#gwr 
cal_cord_gwr = np.asarray(cal_coord_tick)
cal_X_gwr  = np.asarray(cal_X_tick)
cal_y_gwr = np.asarray(cal_y_tick)  
cal_coords_list.append(np.asarray(cal_coord_tick))
cal_X_list.append(np.asarray(cal_X_tick))
cal_y_list.append(np.asarray(cal_y_tick))

In [8]:
 #gwr  
gwr_selector = Sel_BW(cal_cord_gwr, cal_y_gwr, cal_X_gwr,spherical = True)
gwr_bw= gwr_selector.search(bw_min=2)        
gwr_model = GWR(cal_cord_gwr, cal_y_gwr, cal_X_gwr, gwr_bw,spherical = True)
gwr_results = gwr_model.fit()
print(gwr_results.summary())

Model type                                                         Gaussian
Number of observations:                                                  16
Number of covariates:                                                     7

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                            907.560
Log-likelihood:                                                     -55.008
AIC:                                                                124.017
AICc:                                                               146.588
BIC:                                                                882.607
R2:                                                                   0.085
Adj. R2:                                                             -0.525

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------